In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy as sym
import seaborn as sns
from scipy.signal import find_peaks
import myokit
import gc
#from IPython.core.display import HTML
#import pytools.tools as pytools

In [10]:
#For the simulation

def simulation_HH(model, protocol, functions, time, threshold, 
               parameters_hf, parameters_dc, initial_conditions):
    """Detect blocking or not for a set parameters.
    Parameters
    ----------
    model : base model of myokit in .mmt
    protocol : in case the external stimulus needs a protocol
        {[level, start, lenght, period, multiplier], None}, 
        optional 
    functions : list function for external stimulus
        [oscillatory function, constant function]
    time : time simulation
    thresh : 
    max_peaks : integer
    parameters_hf : parameters oscillatory function
        {{'parameter':value},{}}, if the parameter depends on other 
        value so format of value is str
    parameters_dc : parameters constant function
        {{'parameter':value},{}}, if the parameter depends on other 
        value so format of value is str
    initial_conditions: initial conditions
            [m,n,h,voltage]
    
    Returns block, data, num_peaks
    -------
    block --> {0,1} : 0 if there is blocking and 1 if there is not blocking
    data --> data simulation 
    num_peaks --> number of peaks from this simulation
    """
    
    
    mod = myokit.load_model(model)
    
    prot = myokit.Protocol()
    if protocol is not None:
        prot.schedule(protocol[0], protocol[1], protocol[2], protocol[3], protocol[4])
        
    
    ##### Block Stimulus #####  
    ST = mod.get('ST')
    ## oscillatory function ##
    #--------Parameters------#
    for i in parameters_hf:
        var = ST.add_variable(i)
        var.set_rhs(parameters_hf[i])   
    stim_hf = ST.add_variable('stim_hf')
    stim_hf.set_rhs(functions[0])
    ## constant function ##
    #------Parameters-----#
    for i in parameters_dc:
        var = ST.add_variable(i)
        var.set_rhs(parameters_dc[i])
    
    stim_dc = ST.add_variable('stim_dc')
    stim_dc.set_rhs(functions[1])
    ## final stimulus ##
    stim = mod.get('ST.stim') #units [uA/cm^2]
    stim.set_rhs('stim_dc+stim_hf')
    ###########################
    
    ## Initial Conditions ##

    m = mod.get('INA.m')
    m.set_state_value(initial_conditions[0])
    n = mod.get('IK.n')
    n.set_state_value(initial_conditions[1])
    h = mod.get('INA.h')
    h.set_state_value(initial_conditions[2])
    V = mod.get('membrane.V')
    V.set_state_value(initial_conditions[3])
    
    ### Validate model ###
    #print(m.warnings())
    ######################
    
    ## Simulation ##  
    s = myokit.Simulation(mod,prot,path='my-sim.zip')
    
    ### step default ###
    s.set_max_step_size(dtmax=0.005)
    s.set_min_step_size(dtmin=None)
    s.set_tolerance( abs_tol=1e-06 , rel_tol=0.0001 ) 
    ######################## 

    ## Run simulation ##
    df_times = s.run(time)
    ## Find Peaks ##
    dist=(1/(50/s.last_number_of_steps()))
    list_peaks, _ = find_peaks(df_times['membrane.V'],
                               height = threshold, distance = dist)
    del s
    gc.collect()

    return df_times, list_peaks

def parameters_simulation_HH(current, frequency, delta, rho):
    """parameters simulation with base model HH.mmt
    Parameters
    ----------
    delta : slope constant function
    rho : amplitude high frequency
    current : current with slope delta
    frequency: frequency oscillatory function
    value : bool
        plot data in matplotlib figure.
    
    Returns 
    -------
    {0,1} : 0 if there is blocking and 1 if there is not blocking
    data: data simulation in myokit
    num_peaks: peaks 
    """
    """Example
    parameters_simulation(5,0.15,80,4,True)
    """
    model = 'HH.mmt'
    protocol = None
    parameters_hf = {'rho': rho,
                           'f': frequency,
                           'pi':np.pi,
                           'omega_rad':'f*2*pi'}
    oscillatory_function = 'rho*f*1000*cos(omega_rad*t)'
    parameters_dc = {'m': delta,
                           'C':current,
                           't0':'(1/m)+50',
                           'f1':0,
                           'f2': 'C*(t-50)*m',
                           'f3':'piecewise(t<50,f1,f2)'}
    constant_function = 'piecewise(t<t0,f3,C)'
    
    functions = [oscillatory_function,constant_function]
    time = 200
    threshold = 0
    ## this condition ensures starting from a stable state ##
    initial_conditions = [0.0529,0.3176,0.5960,-65.005]
    
    return {"model": model
            , "protocol": protocol
            , "parameters_hf": parameters_hf
            , "oscillatory_function" : oscillatory_function
            , "parameters_dc": parameters_dc
            , "constant_function": constant_function
            , "functions": functions
            , "time": time
            , "threshold": threshold
            , "initial_conditions": initial_conditions
            }


# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()            
 
def run_experiment1(current, frequency,range_delta,range_rho):
    """
    Parameters
    ----------
    
    Returns 
    -------

    """
    
    delta_col=[]
    rho_col=[]
    num_peaks_col=[]
    
    #tracking progress 
    delta_range = np.arange(range_delta[0],range_delta[1],range_delta[2])
    rho_range = np.arange(range_rho[0],range_rho[1],range_rho[2])
    total = len(delta_range)*len(rho_range)
    progress = 0
    for delta in delta_range:
        for rho in rho_range:
            #print('{:.1%}'.format(1.0*progress/total))
            printProgressBar(progress, total, prefix = 'Progress:', suffix = 'Complete', length = 50)
            progress = progress+1
            param = parameters_simulation_HH(current, frequency, delta, rho)
            _, list_peaks = simulation_HH(param["model"]
                                       , param["protocol"]
                                       , param["functions"]
                                       , param["time"]
                                       , param["threshold"]
                                       , param["parameters_hf"]
                                       , param["parameters_dc"]
                                       , param["initial_conditions"])
           
            delta_col.append(delta)
            rho_col.append(rho)
            num_peaks_col.append(len(list_peaks))

    data = pd.DataFrame({'delta':delta_col,'rho':rho_col,'num_peaks':num_peaks_col})
    data['frequency'] = frequency
    data['current'] = current
    data.to_csv(f'DataHH/f{frequency}_I{current}.csv')
    return data

In [11]:
#Visualization for HH

def heatmap_delta_rho_HH(current, frequency):
    """
    Parameters
    ----------
    
    Returns 
    -------

    """

    try: 
        data= pd.read_csv(f"DataHH/f{frequency}_I{current}.csv")
    except:
        print("Missing simulation file" + f"DataHH/f{frequency}_I{current}.csv")
        return

    data = pd.read_csv(f'DataHH/f{frequency}_I{current}.csv')

    data['rho'] = data['rho'].round(2)
    data['delta'] = data['delta'].round(4)
    data=data.pivot("rho", "delta", "num_peaks").astype(float).sort_values('rho',ascending=False)
    plt.figure(figsize = (10,6))
    sns.heatmap(data,cmap="GnBu",annot=False,vmin=0, vmax=5)
    plt.title(rf'$I_0$={current} [$\mu$A/$cm^2$] $\omega$={frequency} [kHz]')
    plt.xlabel(r'$\delta$')
    plt.ylabel(r'$\rho$')
    plt.savefig(f'Figure/HH/I{current}_F{frequency}.png')
    #plt.show()
    plt.close()
    return 

def plot_data(x,y,label_x,label_y,label_plot,title,data):
    """
    Parameters
    ----------
    
    Returns 
    -------

    """
    x = data[x]
    y = data[y]
    plt.figure(figsize = (16,5))
    plt.plot(x,y,label= label_plot)
    plt.title(title)
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.legend()
    plt.show()

def plot_single_simulation_HH(current, frequency, delta, rho):

    param = parameters_simulation_HH(current, frequency, delta, rho)
    df_times, list_peaks = simulation_HH(param["model"]
                                       , param["protocol"]
                                       , param["functions"]
                                       , param["time"]
                                       , param["threshold"]
                                       , param["parameters_hf"]
                                       , param["parameters_dc"]
                                       , param["initial_conditions"])
    MAXPEAKS = 0
    #plot#
    x = df_times['environment.t']
    y = df_times['membrane.V']
    plt.figure(figsize = (16,5))
    plt.plot(x,y,color = 'green')
    for i in list_peaks:
        plt.plot(df_times['environment.t'][i], df_times['membrane.V'][i],
                 label = str(round(df_times['membrane.V'][i],2))+' [mV]', marker='o', color = 'red')
    plt.title('Action Potential')
    plt.xlabel('Time [ms]')
    plt.ylabel('Voltage [mV]')
    plt.legend()
    plt.show()
    #if func[1] != '0':
    plot_data('environment.t','ST.stim_dc','time','[uA/cm^2]','stimulus','Constant Stimulus',df_times)
       #if func[0] != '0':
       #    plot_data('environment.t','ST.stim_hf','time','[uA/cm^2]','stimulus','Oscillatory Stimulus',data)
    plt.close()
    return int(len(list_peaks) > MAXPEAKS), df_times, len(list_peaks)


def generate_polynomial_HH(current, frequency, degree_fit_polynomial, range_delta):
    """
    Parameters
    ----------
    
    Returns 
    -------

    """
    data= pd.read_csv(f"DataHH/f{frequency}_I{current}.csv")
    #this is a patch to use old simulations, remove later
    data.columns = ["index0","index","delta","rho","num_peaks","frequency","current"]
    data_new = data[data['num_peaks']==0]

    rho = data_new['rho'].unique()
    delta = []

    for r in rho:
        maximo = data_new[data_new['rho']==r]['delta'].max()
        delta.append(maximo)
    coef_p = np.polyfit(delta, rho, degree_fit_polynomial)
    
    def polinomio(x):
        f = 0
        for i in list(np.arange(0,degree_fit_polynomial+1)):
            f=f+coef_p[i]*(x**(degree_fit_polynomial+1-(i+1)))
        return f
    plot_delta = list(np.arange(range_delta[0],range_delta[1],range_delta[2]))
    plt.plot(plot_delta,[polinomio(i) for i in plot_delta],
             label = rf'{frequency} $[kHz]$ - {current} $[\mu A/cm^2]$')
    return 

def approach_curve_HH(list_current, frequency, degree_fit_polynomial, range_delta, range_rho):
    """
    Parameters
    ----------
    
    Returns 
    -------

    """
    for current in list_current:
        generate_polynomial_HH(current, frequency, degree_fit_polynomial, range_delta)
    plt.xlim(range_delta[0], range_delta[1])
    plt.ylim(range_rho[0], range_rho[1])
    plt.xlabel(r'$\delta$')
    plt.ylabel(r'$\rho$')
    plt.title('Approach Curve')
    plt.legend()
    plt.savefig(f'Figure/HH/Approach_curve_{frequency}kHz_{degree_fit_polynomial}pol.png')
    plt.close()
    return


In [12]:
## Experiment 2: Simulation for FHN model
def simulation_FHN(model, protocol, func, time, threshold , parameters_hf, parameters_dc, initial):
    """Detect blocking or not for a set parameters.
    Parameters
    ----------
    mod : base model of myokit in .mmt
    prot : in case the external stimulus needs a protocol
        {[level, start, lenght, period, multiplier], None}
    func : list function for external stimulus
        [oscillatory function, constant function]
    time : time simulation
    peaks : find peaks (action potential)
            [height, distance]
    max_peaks : integer
    par_o : parameters oscillatory function
        {{'parameter':value},{}}, if the parameter depends on other 
        value so format of value is str
    par_c : parameters constant function
        {{'parameter':value},{}}, if the parameter depends on other 
        value so format of value is str
    initial: initial conditions
            [v,w]
    value : bool
        plot data in matplotlib figure.
    
    Returns 
    -------
    {0,1} : 0 if there is blocking and 1 if there is not blocking
    d: data simulation in myokit
    num_peaks: peaks with only HF or HF+AMP
    """
    
    mod = myokit.load_model(model)
    prot = myokit.Protocol()
    if protocol is not None:
        prot.schedule(protocol[0],protocol[1],protocol[2],protocol[3],protocol[4])

    ##### Block Stimulus #####  
    ST = mod.get('ST')
    ## oscillatory function ##
    #--------Parameters------#
    for i in parameters_hf:
        var = ST.add_variable(i)
        var.set_rhs(parameters_hf[i])
        
    stim_hf = ST.add_variable('stim_hf')
    stim_hf.set_rhs(func[0])
    ## constant function ##
    #------Parameters-----#
    for i in parameters_dc:
        var = ST.add_variable(i)
        var.set_rhs(parameters_dc[i])
    
    stim_dc = ST.add_variable('stim_dc')
    stim_dc.set_rhs(func[1])
    
    ## final stimulus ##
    stim = mod.get('ST.stim') #units [uA/cm^2]
    stim.set_rhs('stim_hf+stim_dc')
    ###########################
    membrane = mod.get('membrane')
    V_prom = membrane.add_variable('V_prom')
    V_prom.set_rhs('V-ST.rho*sin(ST.omega*environment.t)')
    
    ## Initial Conditions ##
    v = mod.get('membrane.V')
    v.set_state_value(initial[0])
    w = mod.get('membrane.W')
    w.set_state_value(initial[1])
    
    ### Validate model ###
    #print(m.warnings())
    ######################
    
    ## Simulation ##  
    s = myokit.Simulation(mod,prot)
    
    ### step default ###
    s.set_max_step_size(dtmax=0.005)
    s.set_min_step_size(dtmin=None)
    s.set_tolerance( abs_tol=1e-06 , rel_tol=0.0001 ) 
    ########################
     
    ## Run simulation ##
    df_times = s.run(time)
    dist=(1/(50/s.last_number_of_steps()))
    val = 50
    for vm in df_times['environment.t']:
        if vm >=50:
            val=vm
            break
    ## find peaks HF
    list_peaks, _ = find_peaks(df_times['membrane.V_prom'][df_times['environment.t'].index(val)+1:], height = threshold,
                               distance = dist)
  
    return df_times, list_peaks


def parameters_simulation_FHN(current,frequency,delta,rho):
    """parameters simulation with base model HH.mmt
    Parameters
    ----------
    delta : slope constant function
    rho : amplitude high frequency
    current : current with slope delta
    frequency: frequency oscillatory function
    value : bool
        plot data in matplotlib figure.
    
    Returns 
    -------
    {0,1} : 0 if there is blocking and 1 if there is not blocking
    d: data simulation in myokit
    num_peaks: peaks with only HF, only AMP or HF+AMP
    """
    model = 'FHN.mmt'
    protocol = None
    parameters_hf = {'rho': rho,
                           'pi':np.pi,
                           'omega':frequency}
    oscillatory_function = 'rho*omega*cos(omega*t)'
    parameters_dc = {'m': delta,
                           'C':current,
                           't0':'(1/m)+50',
                           'f1':0,
                           'f2': '(t-50)*m*C',
                           'f3':'piecewise(t<50,f1,f2)'}
    constant_function = 'piecewise(t<t0,f3,C)'
    functions = [oscillatory_function,constant_function]
    time = 200 # (current/delta)+150
    
    threshold = 1 
    ## this condition ensures starting from a stable state ##
    initial_conditions = [-1.12,-0.65]
    ## simulation ##
    return {"model": model
           , "protocol": protocol
           , "parameters_hf": parameters_hf
           , "oscillatory_function": oscillatory_function
           , "parameters_dc": parameters_dc
           , "constant_function": constant_function
           , "functions": functions
           , "time": time
           , "threshold": threshold
           , "initial_conditions": initial_conditions
           }



    #return bloqueo, data, num_peaks, peaks



def run_experiment2(current, frequency, range_delta, range_rho):
    delta_col = []
    rho_col = []
    num_peaks_col = []
    
    #tracking progress 
    delta_range = np.arange(range_delta[0],range_delta[1],range_delta[2])
    rho_range = np.arange(range_rho[0],range_rho[1],range_rho[2])
    total = len(delta_range)*len(rho_range)
    progress = 0
    for delta in delta_range:
        for rho in rho_range:
            #print('{:.1%}'.format(1.0*progress/total))
            printProgressBar(progress, total, prefix = 'Progress:', suffix = 'Complete', length = 50)
            progress = progress+1
            param= parameters_simulation_FHN(current,frequency,delta,rho)
            _, list_peaks = simulation_FHN(param["model"]
                                       , param["protocol"]
                                       , param["functions"]
                                       , param["time"]
                                       , param["threshold"]
                                       , param["parameters_hf"]
                                       , param["parameters_dc"]
                                       , param["initial_conditions"])
            delta_col.append(delta)
            rho_col.append(rho)
            num_peaks_col.append(len(list_peaks))
    data = pd.DataFrame({'delta': delta_col,'rho': rho_col,'num_peaks': num_peaks_col})
    data['frequency'] = frequency
    data['current'] = current
    data.to_csv(f'DataFHN/f{frequency}_I{current}.csv')
    return data

In [13]:
#Visualization for FHN, with some work the visualizations could be merged with HH
def heatmap_delta_rho_FHN(current, frequency):
    try: 
        data= pd.read_csv(f"DataFHN/f{frequency}_I{current}.csv")
    except:
        print("Missing simulation file" + f"DataFHN/f{frequency}_I{current}.csv")
        return

    data['rho'] = data['rho'].round(2)
    data['delta'] = data['delta'].round(4)
    
    data=data.drop(columns=['frequency','current']).drop_duplicates()
    plt.figure(figsize = (10,6))
    data=data.pivot("rho", "delta", "num_peaks").astype(float).sort_values('rho',ascending=False)
    graf = sns.heatmap(data,cmap="GnBu",annot=False,vmin=0, vmax=5)
    plt.title(rf'$I_0$={current} $\omega$={frequency} [kHz]')
    plt.xlabel(r'$\delta$')
    plt.ylabel(r'$\rho$')
    plt.savefig(f'Figure/FHN/I{current}_F{frequency}.png')
    #plt.show()
    plt.close()
    return 

def plot_single_simulation_FHN(current, frequency, delta, rho):

    param= parameters_simulation_FHN(current,frequency,delta,rho)
    df_times, _ = simulation_FHN(param["model"]
                                       , param["protocol"]
                                       , param["functions"]
                                       , param["time"]
                                       , param["threshold"]
                                       , param["parameters_hf"]
                                       , param["parameters_dc"]
                                       , param["initial_conditions"])
    marker_flag =0
    delay = [30]
    
    val = 50
    for vm in df_times['environment.t']:
        if vm >=50:
            val=vm
            break

    x = df_times['environment.t']
    w = df_times['membrane.W']
    y = df_times['membrane.V_prom']
    #plt.figure(figsize = (16,5))
    plt.plot(x,y,color='royalblue')

    #makes a mark at time  = val
    for i in delay:
        if marker_flag == 0:
            plt.plot(df_times['environment.t'][i+df_times['environment.t'].index(val)], round(df_times['membrane.V_prom'][i+df_times['environment.t'].index(val)],2),
                marker='x',label = str(round(df_times['membrane.V_prom'][i+df_times['environment.t'].index(val)],2))+' [mV]',color='green')
        else :
            plt.plot(df_times['environment.t'][i+df_times['environment.t'].index(val)], round(df_times['membrane.V_prom'][i+df_times['environment.t'].index(val)],2),
                marker='o',label = str(round(df_times['membrane.V_prom'][i+df_times['environment.t'].index(val)],2))+' [mV]', color='green')

    plt.title(rf'Action Potential - $\delta$:{delta} - $\rho$:{rho}')
    plt.xlabel('time [ms]')
    plt.ylabel('voltage [mV]')
    plt.close()
    return


def generate_polynomial_FHN(current, frequency, degree_fit_polynomial, range_delta):
    data= pd.read_csv(f"DataFHN/f{frequency}_I{current}.csv")
    #this is a patch to use old simulations, remove later
    data.columns = ["index0","index","delta","rho","num_peaks","frequency","current"]
    data_new = data[data['num_peaks']==0]
    rho = data_new['rho'].unique()
    delta = []

    for r in rho:
        maximo = data_new[data_new['rho']==r]['delta'].max()
        delta.append(maximo)
    coef_p = np.polyfit(delta,rho,degree_fit_polynomial)
    
    def polinomio(x):
        f = 0
        for i in list(np.arange(0,degree_fit_polynomial+1)):
            f=f+coef_p[i]*(x**(degree_fit_polynomial+1-(i+1)))
        return f
    plot_delta = list(np.arange(range_delta[0],range_delta[1],range_delta[2]))
    plt.plot(plot_delta,[polinomio(i) for i in plot_delta],
             label = rf'{frequency} $[kHz]$ - {current}')
    return 

def approach_curve_FHN(set_current, frequency, degree_fit_polynomial, range_delta, range_rho):
    for current in set_current:
        generate_polynomial_FHN(current, frequency, degree_fit_polynomial, range_delta)
    plt.xlim(range_delta[0], range_delta[1])
    plt.ylim(range_rho[0], range_rho[1])
    plt.xlabel(r'$\delta$')
    plt.ylabel(r'$\rho$')
    plt.title('Approach Curve')
    plt.legend()
    plt.savefig(f'Figure/FHN/Approach_curve_{frequency}kHz_{degree_fit_polynomial}pol.png')
    plt.close()
    return 



In [14]:
def main():
    # Execute experiment 1: amplitude of HF term vs slope for the HH model

    #current, frequency = 40, 10 
    current, frequency = 80, 10 
    #current, frequency = 120, 10
    
    #parameter mesh
    range_delta = [0.0025,0.1275,0.0025]
    range_rho = [0.01,0.3,0.01]
    #range_delta = [0.0025,0.1275,0.01]
    #range_rho = [0.01,0.3,0.05]

    # Generate simulation    
    output = run_experiment1(current,frequency,range_delta,range_rho) 

    # Generate plots
    #heatmap_delta_rho_HH(current, frequency) ### ACTUALIZAR 'num_peaks_HF' - 'num_peaks_HF_DC' ###
    #plot_single_simulation_HH(current, frequency, delta = 0.05, rho = 0.06)
    #approach_curve_HH([40,80,120], frequency, degree_fit_polynomial = 6, range_delta= range_delta, range_rho = range_rho)

    #Execute experiment 2: FHN
    #current, frequency = 0.15, 5
    current, frequency = 0.3, 5
    #current, frequency = 0.4, 5
    #current, frequency = 0.5, 5
    range_delta = [0.01,0.195,0.005]
    range_rho = [0.1,1.55,0.05]
    #range_delta = [0.01,0.2,0.05]
    #range_rho = [0.1,0.75,0.1]
    
    # Generate simulation  
    #output = run_experiment2(current,frequency,range_delta,range_rho) 
    
    # Generate plots
    #heatmap_delta_rho_FHN(current, frequency)
    #plot_single_simulation_FHN(current, frequency, delta = 0.11, rho = 0.5)
    #looks weird, maybe I broke something
    #approach_curve_FHN([0.15,0.3,0.4,0.5], frequency, degree_fit_polynomial = 4, range_delta= range_delta, range_rho = range_rho)


main()

FileNotFoundError: [WinError 206] El nombre del archivo o la extensión es demasiado largo: 'c:\\miniconda3\\lib\\site-packages\\myokit\\_bin\\sundials-win-vs\\lib'

## 